In [1]:
import pandas as pd
import numpy as np
import os
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [2]:
# defining exact paths
# Go up two levels from 'notebooks/modeling' to project root
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), '../..'))

RAW_DATA_PATH = os.path.join(BASE_DIR, 'data', 'raw', 'WA_Fn-UseC_-Telco-Customer-Churn.csv')
PROCESSED_DATA_PATH = os.path.join(BASE_DIR, 'data', 'processed')

# creating the processed folder if it doesn't exist
os.makedirs(PROCESSED_DATA_PATH, exist_ok=True)

print(f"Reading data from: {RAW_DATA_PATH}")
print(f"Saving data to: {PROCESSED_DATA_PATH}")

Reading data from: d:\IIT ACADEMICS\SECOND YEAR\Machine Learning - CM2604\telco-customer-churn-analysis\data\raw\WA_Fn-UseC_-Telco-Customer-Churn.csv
Saving data to: d:\IIT ACADEMICS\SECOND YEAR\Machine Learning - CM2604\telco-customer-churn-analysis\data\processed


### LOADING DATA

In [3]:
df = pd.read_csv(RAW_DATA_PATH)

In [4]:
pd.set_option('display.max_columns', None)
display(df.head(2))

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


### DROP UNNECESSARY COLUMNS

In [5]:
df.drop(columns=['customerID'], inplace=True)

### FIX: TotalCharges (Coverting to numeric and imputing missing values)

In [8]:
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

#df['TotalCharges'].fillna(0, inplace=True)
df['TotalCharges'] = df['TotalCharges'].fillna(0)

# filling NaN values with 0 safely (pandas update)